### 코스피 분석 연습
* http://www.paxnet.co.kr/stock/sise/KOSPI?wlog_rpax=KOSPI
* 팍스넷의 데이터 웹크롤링을 통한 코스피 일별 거래정보 획득


In [22]:
import sqlite3
import requests
from bs4 import BeautifulSoup
dbname='paxstock.db'

In [23]:
#['20190710', '2058.78', '2061.55', '2068.20', '2056.48', 1, '6.75', '+0.33', '355677']
def maketable():
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        sql="""create table kospi(
        IDX integer primary key,
        EPRICE REAL,
        SPRICE REAL,
        HPRICE REAL,
        LPRICE REAL,
        UDCODE integer,
        UDOWN  REAL,
        UDRATE  REAL,
        TRQTY integer )"""
        cur.execute(sql)
        conn.commit()
        cur.close()

In [24]:
maketable()

OperationalError: table kospi already exists

In [25]:
def insertdb(data):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #종가	시가	고가	저가	전일비	등락률	거래량(천주)
        sql="""
        insert into kospi 
        (IDX,EPRICE,SPRICE,HPRICE,LPRICE,UDCODE,UDOWN,UDRATE,TRQTY)
        values(?,?,?,?,?,?,?,?,?)
        """
        idx=cur.execute(sql,data)
        sql_title='select last_insert_rowid()'
        resultid=cur.execute(sql_title).fetchall()
        print('lastid in:',resultid[0])
        return resultid[0][0]
        

In [26]:
d=['20190710', '2058.78', '2061.55', '2068.20', '2056.48', 1, '6.75', '+0.33', '355677']
insertdb(d)

IntegrityError: UNIQUE constraint failed: kospi.IDX

In [27]:
def Mycrawl(soup):
    tgt=soup.select('div.table-scroll table.table-data tbody > tr ')
    #, row_factory=sqlite3.Row
    with sqlite3.connect(dbname) as conn:
        for t in tgt:
            data=[]
            idx=t.select('td > strong')[0].get_text() #print(idx)[0].get_text()
            idx=idx.replace('.','')# 날짜를 키값으로 사용하기
            #print("row:",idx,end="\n")
            #종가	시가	고가	저가	전일비	등락률	거래량(천주)
            eprice=t.select('td.a-right')[0].get_text().replace(',','')
            sprice=t.select('td.a-right')[1].get_text().replace(',','')
            hprice=t.select('td.a-right')[2].get_text().replace(',','')
            lprice=t.select('td.a-right')[3].get_text().replace(',','')
            
            udrate=t.select('td.a-right')[5].get_text().strip().replace('%','')
            trqty=t.select('td.a-right')[6].get_text().replace(',','')
            
            updown=t.select('td.a-right > span.rise')
            if(updown):
                udown=updown[0].get_text()
                udcode=1
                udown=udown.replace('상향','').strip()
            else:
                udcode=-1
                udown=t.select('td.a-right > span.fall')[0].get_text()
                udown=udown.replace('하향','').strip()
            data=[idx,eprice,sprice,hprice,lprice,udcode,udown,udrate,trqty]
            print(data,end="\n")
            
            #title=t.select('a.movie')[0].get_text()     #print(title,end="\n")
            
 

In [28]:
for i in range(1,2):
    url='http://www.paxnet.co.kr/stock/sise/siseJisuDataListAjax?searchType=kospi&currentPageNo='
    url+=str(i)
    url+='&searchSort=&searchExchange=KSE&searchTerm=&searchUpDown='
    result=requests.get(url)
    src=result.content
    #print(result.content)
    soup=BeautifulSoup(src,'html.parser')
    #print(url,end="\n")
    Mycrawl(soup)

['20190710', '2058.78', '2061.55', '2068.20', '2056.48', 1, '6.75', '+0.33', '355677']
['20190709', '2052.03', '2070.35', '2075.30', '2052.03', -1, '12.14', '-0.59', '440122']
['20190708', '2064.17', '2091.91', '2092.54', '2063.78', -1, '46.42', '-2.20', '515950']
['20190705', '2110.59', '2111.00', '2115.43', '2096.23', 1, '1.86', '+0.09', '459391']
['20190704', '2108.73', '2102.36', '2108.80', '2088.97', 1, '12.71', '+0.61', '446932']
['20190703', '2096.02', '2120.75', '2127.71', '2093.32', -1, '26.00', '-1.23', '522709']
['20190702', '2122.02', '2122.07', '2130.86', '2112.52', -1, '7.72', '-0.36', '460830']
['20190701', '2129.74', '2147.24', '2147.24', '2126.87', -1, '0.88', '-0.04', '412291']
['20190628', '2130.62', '2135.06', '2139.24', '2123.97', -1, '3.70', '-0.17', '537274']
['20190627', '2134.32', '2120.73', '2141.27', '2118.22', 1, '12.47', '+0.59', '644900']
['20190626', '2121.85', '2120.20', '2128.25', '2115.72', 1, '0.21', '+0.01', '736851']
['20190625', '2121.64', '2125.74